In [68]:
import fitz
import requests
import cv2
import imutils
import io
import pytesseract
import camelot.io as camelot
import pandas as pd
import numpy as np
from collections import Counter
from textblob import TextBlob
from PIL import Image

# PDF Extractor

In [39]:
# Extracting words from "dissertation.pdf"

doc = fitz.open('data/dissertation.pdf')
text = "".join(page.get_text("text") for page in doc)
words = pd.Series(text.split())
words.value_counts().head(30)

the         2640
.           2323
of          1580
to          1384
and         1174
a            977
in           694
is           612
for          524
that         507
be           431
data         407
The          361
are          349
with         337
as           307
students     259
can          257
this         247
course       233
it           222
or           221
on           210
an           190
their        182
was          176
by           171
Data         171
not          168
I            155
dtype: int64

In [40]:
# Extracting tabular data from "calendar.pdf"

tables = camelot.read_pdf('data/calendar.pdf')
df = tables[0].df
df

,0,1,2,3,4
0,Fall,2020 Fall Term,,2021 Fall Term,
1,First Day of Classes,Tue,September 1,Tue,August 31
2,Labor Day - Classes Will Meet,Mon,September 7,,
3,Labor Day - Classes Suspended,,,Mon,September 6
4,Last day to add or drop courses,Tue,September 15,Tue,September 14
...,...,...,...,...,...
70,Summer Session - 10 week - classes begin,Mon,June 7,Mon,June 6
71,Last day to add or drop courses,Wed,June 16,Wed,June 15
72,Last day to change registration or withdraw fr...,Thur,July 15,Thur,July 14
73,Final Exams,Fri,August 13,Fri,August 12


In [41]:
# Extracting tabular data from "GingerChocolateChipCookies.pdf"
cookie_tables = camelot.read_pdf('data/GingerChocolateChipCookies.pdf')
cookie_df = cookie_tables[0].df
cookie_df

,0,1,2,3,4
0,4,c,oats,,
1,3.0,c,ﬂour 2.5?,,
2,2,t,soda,,
3,1.0,t,salt,,
4,2,t,ginger powder,,
5,0.5,t,hot red pepper powder,,
6,,,,,
7,4,,eggs,,
8,2,c,sugar + 2T,,
9,2,T,molasses,,


# Reddit Image Transcriber

In [73]:
# Example version
# Set a User Agent to avoid being blocked
data = requests.get("https://www.reddit.com/r/comics/.json", headers = {'User-agent': 'your bot 0.1'}).json()

i = 0
for element in data['data']['children']:
    # only do this 5 times because this is just for demonstration purposes and it would print a LOT out otherwise
    print('\n')
    if i > 5:
        break
    if 'url' in element['data'] and (element['data']['url'].endswith('.png') or element['data']['url'].endswith('.jpg')):
        url = element['data']['url']
        title = element['data']['title']
        response = requests.get(url)
        img = Image.open(io.BytesIO(response.content))
        text = pytesseract.image_to_string(img)
        words = pd.Series(text.split())
        print(words.value_counts())
        blob = TextBlob(text)
        print(blob.noun_phrases)

        # Sorted with pandas
        pd.Series(blob.word_counts).sort_values().tail(30)
        
        blob = TextBlob(text)
        print(blob.sentiment.polarity)
        # Value from -1 to 1 about how positive/negative the post's words are
        
        i += 1



ROCK?               1
LOVE                1
MRLOVENSTEIN.COM    1
BEST!               1
DUDE,               1
|                   1
LIKE                1
YEAH!               1
You                 1
>                   1
!                   1
HELL                1
ROCK'S              1
ROCK!               1
THE                 1
dtype: int64
['dude', 'love rock', 'hell yeah', 'like rock', 'rock', 'the best', 'mrlovenstein.com']
0.98828125




<ipython-input-73-394aee24b8b4>:16: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  words = pd.Series(text.split())
<ipython-input-73-394aee24b8b4>:22: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(blob.word_counts).sort_values().tail(30)


Series([], dtype: int64)
[]
0.0


@REMI_LASCAULT    1
dtype: int64
['remi_lascault']
0.0


DYING!        1
IT’S          1
YOU           1
OKAY,         1
CALL!         1
UNDERSTAND    1
TO            1
|             1
CALL          1
HAVE          1
\'M           1
911!          1
dtype: int64
["\\ 'm", 'dying', 'call', 'you have to call', 'it', '’ s', 'okay', 'understand']
0.5


VVE          1
(M           1
THE          1
.            1
FISH         1
GOT          1
LIFES        1
[ngewe       1
BURDENS!)    1
DROWNING     1
LINES        1
FISH.        1
DRAWN        1
OF           1
IN           1
POORLY       1
YOU'RE       1
MAN,         1
A            1
CC           1
BURDENS      1
MY           1
dtype: int64
['[ ngewe', 'cc', 'drowning in the burdens of my lifes man', "you're", 'fish', 'vve got fish burdens', 'poorly drawn lines']
-0.4


JUST         2
T            2
DON          2
BLADE!       1
COME         1
LEAVE        1
WITH         1
wHOA!        1
YET!         1
I'M   

In [87]:
# Trying out sentiment analysis on a subreddit of my choice
# Set a User Agent to avoid being blocked
# Another option: https://www.reddit.com/r/poodles/ https://www.reddit.com/r/doodledogs/.json
data = requests.get("https://www.reddit.com/r/Infographics/.json", headers = {'User-agent': 'your bot 0.1'}).json()
reddit_dict = {'Title' : [], 'URL' : [], 'Sentiment Rating' : []}

for element in data['data']['children']:
    # only do this 5 times because this is just for demonstration purposes and it would print a LOT out otherwise
    if 'url' in element['data'] and (element['data']['url'].endswith('.png') or element['data']['url'].endswith('.jpg')):
        url = element['data']['url']
        title = element['data']['title']
        response = requests.get(url)
        img = Image.open(io.BytesIO(response.content))
        text = pytesseract.image_to_string(img)
        words = pd.Series(text.split())
        blob = TextBlob(text)
        reddit_dict['Title'].append(title)
        reddit_dict['URL'].append(url)
        reddit_dict['Sentiment Rating'].append(blob.sentiment.polarity)
        # Value from -1 to 1 about how positive/negative the post's words are

reddit_df = pd.DataFrame(reddit_dict)
reddit_df

<ipython-input-87-d11fc6a36e4a>:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  words = pd.Series(text.split())


,Title,URL,Sentiment Rating
0,Map of Computer Science,https://i.redd.it/3oa8m9wkegy61.png,0.069167
1,Comparing Gun Control Measures to Gun-Related ...,https://i.imgur.com/vM83vt5.png,0.112121
2,How to Solve A Rubik's Cube,https://i.redd.it/zn5r2d9fe9y61.jpg,0.132808
3,Death troll in Afghanistan war!,https://i.redd.it/cs0haqvhhby61.png,-0.131250
4,How to make 2 piece silicone rubber mold,https://i.redd.it/utwkfvhznfy61.jpg,0.078799
5,Laser Cutter,https://i.redd.it/mssr6por8iy61.jpg,-0.200000
6,"2,000 Years of Economic History in One Chart",https://i.redd.it/mat1bnb25ay61.jpg,0.131250
7,Which countries is Apple worth more than? (By ...,https://i.redd.it/ly41ipyhgay61.jpg,0.185522
8,Create you own crypto wallet,https://i.redd.it/pcaijmyq2ay61.png,0.143137
9,Key Events in U.S. History that Defined Genera...,https://i.redd.it/3wluh713rxx61.jpg,0.002546


# Face Finding

In [43]:
# Human face finding
# Taken from tutorial at 
# https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html#face-detection

face_cascade = cv2.CascadeClassifier('xml/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('xml/haarcascade_eye.xml')

def face_detect(imgpath, face_cascade, eye_cascade):
    img = cv2.imread(imgpath)
    img = imutils.resize(img, width=800)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)

    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
face_detect('data/me.jpg', face_cascade, eye_cascade)

In [44]:
# Cat face finding
# Tutorial: https://techtutorialsx.com/2021/04/11/python-opencv-detect-cat-faces/

face_cascade = cv2.CascadeClassifier('xml/haarcascade_frontalcatface.xml')

def cat_face_detect(imgpath, face_cascade):
    img = cv2.imread(imgpath)
    img = imutils.resize(img, width=800)
    
    faces = face_cascade.detectMultiScale(img)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0))
        
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
cat_face_detect('data/cat.jpg', face_cascade)